In [1]:
import pandas as pd
import numpy as np
from requests import post as requests_post
import wrds
import os
from statsmodels.sandbox.regression import gmm

In [2]:
# Import data

def request_wrds(path: str,
                 start_date: pd.Period,
                 end_date: pd.Period) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    with wrds.Connection() as db:
        df_s12 = db.raw_sql(f'''
            SELECT *
            FROM tfn.s12
            WHERE fdate >= '{start_date}' AND fdate <= '{end_date}'
        ''', date_cols=['fdate'])
        df_s12.to_csv(f'{path}s12.csv')

        df_s34 = db.raw_sql(f'''
            SELECT *
            FROM tfn.s34
            WHERE fdate >= '{start_date}' AND fdate <= '{end_date}'
        ''', date_cols=['fdate'])
        df_s34.to_csv(f'{path}s34.csv')

        df_ffm = db.raw_sql(f'''
            SELECT *
            FROM ff.factors_monthly
            WHERE date >= '{start_date}' AND date <= '{end_date}'
        ''', date_cols=['date'])
        df_ffm.to_csv(f'{path}ffm.csv')

        df_security = db.raw_sql(f'''
            SELECT *
            FROM ff.factors_monthly
            WHERE date >= '{start_date}' AND date <= '{end_date}'
        ''', date_cols=['date'])
        df_security.to_csv(f'{path}security.csv')

        df_fundamental = db.raw_sql(f'''TODO''', date_cols=['date'])
        df_fundamental.to_csv(f'{path}fundamental.csv')

        return df_s12, df_s34, df_ffm, df_security, df_fundamental


# def wrds_beta(api_key: str, begin_date: str, end_date: str):
#     url = 'https://wrds-api.wharton.upenn.edu/v0.1/research/betas/'
#     headers = {
#         'Authorization': f'Token {api_key}',
#     }
#     data = {
#         'company_identifier': 'PERMNO',
#         'begin_date': begin_date,
#         'end_date': end_date,
#         'freq': 'm',
#         'estwindow': 5 * 12,
#         'minwindow': 1 * 12,
#         'risk_model': 'ff3',
#         'return_type': 'r'
#     }
#     result = requests_post(
#         url,
#         headers=headers,
#         json=data,
#     )
#     df = pd.DataFrame(result.json()['betas'], columns=result.json()['betas_cols'])
#     return df


def load_wrds(path: str,
              start_date: pd.Period,
              end_date: pd.Period
              ) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    try:
        df_s12 = pd.read_csv(f'{path}s12.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_s12(df_s12)

        df_s12type5 = pd.read_csv(f'{path}s12type5.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_s12type5(df_s12type5)

        df_s34 = pd.read_csv(f'{path}s34.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_s34(df_s34)

        df_ffm = pd.read_csv(f'{path}ffm.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_ffm(df_ffm)

        df_beta = pd.read_csv(f'{path}beta.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_beta(df_beta)

        df_security = pd.read_csv(f'{path}security.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_security(df_security)

        df_fundamental = pd.read_csv(f'{path}fundamental.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_fundamental(df_fundamental)

        return df_s12, df_s12type5, df_s34, df_ffm, df_beta, df_security, df_fundamental
    except FileNotFoundError:
        return request_wrds(path, start_date, end_date)


def clean_imports(df_s12,
                  df_s12type5,
                  df_s34,
                  df_ffm,
                  df_beta,
                  df_security,
                  df_fundamental,
                  start_date,
                  end_date
                  ) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):

    df_s12_clean = clean_s12(df_s12, start_date, end_date)
    log_clean_s12(df_s12_clean)

    df_s12type5_clean = clean_s12type5(df_s12type5, start_date, end_date)
    log_clean_s12type5(df_s12type5_clean)

    df_s34_clean = clean_s34(df_s34, start_date, end_date)
    log_clean_s34(df_s34_clean)

    df_ffm_clean = clean_ffm(df_ffm, start_date, end_date)
    log_clean_ffm(df_ffm_clean)

    df_beta_clean = clean_beta(df_beta, start_date, end_date)
    log_clean_beta(df_beta_clean)

    df_security_clean = clean_security(df_security, start_date, end_date)
    log_clean_security(df_security_clean)

    # TODO: df_fundamental_clean = clean_fundamental(df_fundamental, start_date, end_date)
    df_fundamental_clean = df_fundamental
    # log_clean_fundamental(df_fundamental_clean)

    return df_s12_clean, df_s12type5_clean, df_s34_clean, df_ffm_clean, df_beta_clean, df_security_clean, df_fundamental_clean


def clean_s12(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'fundno',
        'rdate',
        'cusip',
        'shares'
    ]
    df['rdate'] = df['rdate'].apply(fix_date, args=(start_date, end_date))
    return (df[columns]
            .dropna(how='any', subset=['fundno', 'shares', 'cusip'])
            .drop_duplicates(subset=['fundno', 'rdate', 'cusip'], keep='last')
            .rename(columns={
                'rdate': 'date',
                'fundno': 'inv_id',
                'cusip': 'asset_id'
            })
            .set_index(['inv_id', 'date', 'asset_id']))


def clean_s12type5(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    return (df
            .assign(fdate=df['fdate'].apply(fix_date, args=(start_date, end_date)))
            .dropna(how='any', subset=['fundno', 'fdate'])
            .rename(columns={'fdate': 'date', 'fundno': 'inv_id'})
            .set_index(['inv_id', 'date']))


def clean_s34(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'mgrno',
        'rdate',
        'typecode',
        'cusip',
        'shares',
        'backup_holding',
        'backup_me'
    ]
    df['rdate'] = df['rdate'].apply(fix_date, args=(start_date, end_date))
    df['backup_holding'] = df['shares'] * df['prc']
    df['backup_me'] = df['shrout2'] * df['prc'] * 1000
    return (df[columns]
            .dropna(how='any', subset=['cusip', 'shares'])
            .drop_duplicates(subset=['mgrno', 'rdate', 'cusip'])
            .rename(columns={
                'rdate': 'date',
                'mgrno': 'inv_id',
                'cusip': 'asset_id'
            })
            .set_index(['inv_id', 'date', 'asset_id']))


def clean_ffm(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    df['dateff'] = df['dateff'].apply(fix_date, args=(start_date, end_date))
    return (df
            .dropna()
            .rename(columns={'dateff': 'date'})
            .set_index(['date']))


def clean_beta(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns=[
        'PERMNO',
        'DATE',
        'b_mkt',
        'b_smb',
        'b_hml'
    ]
    df['DATE'] = df['DATE'].apply(fix_date, args=(start_date, end_date)) + 6
    return (df[columns]
            .rename(columns={
                'DATE': 'date',
                'PERMNO':'permno'})
            .dropna()
            .drop_duplicates(subset=['date', 'permno'], keep='last')
            .set_index(['date', 'permno']))


def clean_security(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'LPERMNO',
        'cusip',
        'datadate',
        'prccm',
        'cshoq'
    ]
    return (df[columns]
            .rename(columns={
                'LPERMNO': 'permno',
                'cusip': 'asset_id',
                'prccm': 'prc',
                'cshoq': 'shrout',
                'datadate': 'date'})
            .assign(
                shrout=df['cshoq'] * 1000000,
                asset_id=df['cusip'].apply(lambda  x: x[:-1]),
                date=df['datadate'].apply(fix_date, args=(start_date, end_date)))
            .dropna(how='any', subset=['asset_id', 'prc', 'shrout'])
            .drop_duplicates(subset=['date', 'permno'], keep='last')
            .set_index(['date', 'permno']))


def clean_fundamental(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'LPERMNO'
        'cusip',
        'datadate',
        'fyearq',
        'fqtr',
        'fyr'
        'act',
        'at',
        'ceq',
        'che',
        'cogs',
        'csho',
        'dlc',
        'lct',
        'lt',
        'pstk',
        'pstkl',
        'pstkrv',
        'revt',
        'seq',
        'txditc',
        'xint',
        'xsga',
        'sich'
    ]
    df.columns = [col[:-1] if col[-1] == 'q' else col for col in df.columns]
    return (df[columns]
            .rename(columns={
                'LPERMNO': 'permno',
                'cusip': 'asset_id',
                'csho': 'shrout'})
            .assign(
                shrout=df['shrout'] * df['ajex'] * 1000,
                date=df['date'].apply(fix_date, args=(start_date, end_date)),
                na_flag=df['cogs'].isnan() & df['cogs'].isnan() & df['cogs'].isnan(),
                cogs=lambda x: 0 if x['na_flag'] else x['cogs'],
                xint=lambda x: 0 if x['na_flag'] else x['xint'],
                xsga=lambda x: 0 if x['na_flag'] else x['xsga'],
                txditc=df['txditc'].fillna(0),
                seq=lambda x: x['ceq'] + x['pstk'] if np.isnan(x['seq']) else x['seq'])
            .assign(
                seq=lambda x: x['seq'].fillna(x['at'] - x['lt']),
                at=lambda x: x['seq'].fillna(x['lt'] + x['seq']),
                lt=lambda x: x['seq'].fillna(x['at'] - x['seq']),
                lag_date=lambda x: x['date'] + 6,
                preferred=lambda x: x['pstkrv'].fillna(x['pstkl'].fillna(x['pstk'])),
                be=lambda x: np.nan if x['seq'] + x['txditc'] - x['preferred'] <= 0 else x['seq'] + x['txditc'] - x['preferred'],
                operating=lambda x: x['act'] - x['che'] - x['lct'] + x['dlc'],
                at_be=lambda x: x['at'] / x['be'],
                profit=lambda x: (x['revt'] - x['cogs'] - x['xsga'] - x['xint']) / x['be'],
                lag_fyear=df['fyear'].shift(1),
                accruals='df')
            .drop(columns=['ajex', 'lag_fyear'])
            .dropna(how='any', subset=['asset_id', 'prc', 'shrout'])
            .set_index(['date', 'permno']))


def fix_date(date: str, start_date: pd.Period, end_date: pd.Period) -> pd.Period:
    date_converted = pd.Period(date, freq='M')

    if date_converted < start_date or date_converted > end_date:
        return np.NaN
    else:
        return date_converted


# Stocks Monthly

def merge_assets_factors(df_assets: pd.DataFrame, df_factors: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(left=df_assets,
                      right=df_factors,
                      how='inner',
                      left_index=True,
                      right_index=True)
    merged_indexed = (merged
                      .reset_index('permno', drop=True)
                      .set_index('asset_id', append=True)
                      .sort_index())
    log_asset_merge(merged_indexed)
    return merged_indexed

In [3]:
# Manager / Holdings

def match_fund_manager(df_fund: pd.DataFrame, df_manager: pd.DataFrame, df_key: pd.DataFrame) -> pd.DataFrame:
    # TODO
    # df_fund_wkey = df_fund.assign(mgrno=lambda x: df_key.loc[x.index.get_level_values(0), 'mgrcocd'])
    # df_merged = df_manager.merge(df_fund_wkey, how='outer', on='mgrno')
    log_holding_merge(df_manager)
    return df_manager


def merge_holding_factor(df_holding: pd.DataFrame, df_asset: pd.DataFrame) -> pd.DataFrame:
    df_merged = pd.merge(
        left=df_holding,
        right=df_asset,
        how='left',
        left_index=True,
        right_index=True,
        indicator=True
    ).reset_index()

    df_merged = (df_merged
                .assign(
                    ccm_holding=lambda x: x['prc'] * x['shares'],
                    ccm_me=lambda x: x['prc'] * x['shrout'],
                    holding=lambda x: x['backup_holding'] if x['ccm_holding'].isna().any() else x['ccm_holding'],
                    me=lambda x: x['backup_me'] if x['ccm_me'].isna().any() else x['ccm_me'],
                    date=df_merged['date'].apply(lambda x: x.asfreq('Q')))
                 .drop(columns=['ccm_holding', 'backup_holding', 'ccm_me', 'backup_me'])
                .set_index(['inv_id', 'date', 'asset_id'])
                .sort_index())
    log_holding_factor_merge(df_merged)
    return df_merged


def create_outside_asset(df_holding_factor: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    df_inside = df_holding_factor[df_holding_factor['_merge'] == 'both'].drop(columns='_merge')
    df_outside = df_holding_factor[df_holding_factor['_merge'] == 'left_only'].groupby(['inv_id', 'date']).agg({
        'typecode': 'first',
        'holding': 'sum'
    }).assign(asset_id='-1').set_index('asset_id', append=True)
    log_outside_asset(df_outside)
    return df_inside, df_outside


def assetid_all(df_holding: pd.DataFrame, df_asset: pd.DataFrame) -> pd.Series:
    unique_asset_id = df_holding.index.unique('asset_id')
    universe_asset_id = df_asset.index.unique('asset_id')
    intersection = np.intersect1d(unique_asset_id, universe_asset_id, assume_unique=True)
    return pd.Series(intersection)


def assetid_byinv(df_holding: pd.DataFrame) -> pd.Series:
    return df_holding.reset_index('asset_id')['asset_id']


def date_byassetid(df_outside: pd.DataFrame) -> pd.DataFrame:
    idx_holding = (df_outside
                   .reset_index('date')
                   ['date']
                   .groupby('asset_id', group_keys=True)
                   .apply(np.unique))
    df_date_byassetid = idx_holding.copy()
    for asset_id in idx_holding.index:
        holding_dates = idx_holding.loc[asset_id]
        df_date_byassetid.loc[asset_id] = holding_dates

    return (df_date_byassetid
            .drop('-1')
            .explode()
            .reset_index())


def create_household_sector(df_outside: pd.DataFrame) -> pd.DataFrame:
    df_household = (df_outside
                      .groupby(['date', 'asset_id'])
                      .agg({
                        'shares': 'sum',
                        'prc': 'last',
                        'shrout': 'last',
                        'b_mkt': 'last',
                        'b_smb': 'last',
                        'b_hml': 'last',
                        'holding': 'sum',
                        'me': 'last'})
                      .assign(
                        shares=lambda x: x['shrout'] - x['shares'],
                        holding=lambda x: x['me'] - x['holding'],
                        inv_id=-1,
                        typecode=-1)
                    .set_index('inv_id', append=True)
                    .reorder_levels(['inv_id', 'date', 'asset_id']))
    log_household_sector(df_household)
    return df_household

In [247]:
# Data

def calc_inv_aum(df_inside: pd.DataFrame, df_outside: pd.DataFrame) -> pd.DataFrame:
    df_in_aum = (df_inside
                 .groupby(['inv_id', 'date'])
                 .agg({
                   'holding': 'sum',
                   'shares': 'count',
                   'typecode': 'last'})
                 .rename(columns={
                   'holding': 'in_aum',
                   'shares': 'n_holding'}))

    df_out_aum = (df_outside
                  .groupby(['inv_id', 'date'])
                  .agg({'holding': 'sum'})
                  .rename(columns={'holding': 'out_aum'}))

    df_merged = pd.merge(
        left=df_in_aum,
        right=df_out_aum,
        how='outer',
        left_index=True,
        right_index=True)
    
    df_inv_aum = (df_merged
                  .assign(
                    out_aum=lambda x: x['out_aum'].fillna(0),
                    in_aum=lambda x: x['in_aum'].fillna(0),
                    aum=lambda x: x['out_aum'] + x['in_aum']))

    log_inv_aum(df_inv_aum)
    return df_inv_aum


def bin_concentrated_inv(df_inside: pd.DataFrame, df_inv_aum: pd.DataFrame) -> pd.DataFrame:
    cutoff = 50
    diversified_mask = df_inv_aum['n_holding'] >= cutoff
    household_mask = (df_inv_aum['out_aum'] == 0) & (df_inv_aum['in_aum'] == 0)
    
    df_aum_diversified = df_inv_aum[diversified_mask & ~household_mask]
    df_aum_concentrated = df_inv_aum[~diversified_mask & ~household_mask].assign(bin=np.NaN)
    
    inv_type_codes = df_inv_aum['typecode'].unique()
    for type in inv_type_codes:
        type_mask = df_aum_concentrated['typecode'] == type
        df_type = df_aum_concentrated[type_mask]
        n_bins = np.ceil(df_type['n_holding'].sum() / (2 * cutoff)).astype(int)
        
        if n_bins <= 1:
            df_aum_concentrated.loc[type_mask, 'bin'] = 10 * type
        else:
            df_aum_concentrated.loc[type_mask, 'bin'] = pd.qcut(df_type['aum'], q=n_bins, labels=False) + (10 * type)


    df_concentrated_binned = (df_aum_concentrated
                              .reset_index()
                              .groupby(['bin', 'date'])
                              .agg({
                                'in_aum': 'sum',
                                'out_aum': 'sum',
                                'aum': 'sum',
                                'n_holding': 'sum',
                                'typecode': 'last'})
                              .rename_axis(index={'bin': 'inv_id'}))
    df_aum_binned = (pd.concat([df_aum_diversified, df_concentrated_binned])
                     .sort_index()
                     .assign(
                       out_weight=lambda x: x['out_aum'] / x['aum'],
                       in_weight=lambda x: x['in_aum'] / x['aum']))
    
    df_inside_merged = pd.merge(
        left=df_inside,
        right=df_aum_concentrated['bin'],
        how='left',
        left_index=True,
        right_index=True)
                        
    df_inside_binned = (df_inside_merged
                        .reset_index()
                        .assign(bin=lambda x: x['bin'].fillna(x['inv_id']).astype(int))
                        .groupby(['bin', 'date', 'asset_id'])
                        .agg({
                          'holding': 'sum',
                          'prc': 'last',
                          'shrout': 'last',
                          'b_mkt': 'last',
                          'b_smb': 'last',
                          'b_hml': 'last'})
                        .rename_axis(index={'bin': 'inv_id'}))
    
    log_bins(df_inside_binned, df_aum_binned)
    return df_inside_binned, df_aum_binned


def calc_inv_universe(df_holding: pd.DataFrame) -> pd.DataFrame:
    num_quarters = 11
    df_assetid_byinv = assetid_byinv(df_holding).sort_index()
    idx_inv_universe = df_assetid_byinv.index.unique()
    df_inv_universe = pd.DataFrame(index=idx_inv_universe, columns=['inv_universe'])

    def calc_past_quarters(i: int, d: pd.Period) -> np.array:
        prev_date = d - num_quarters
        asset_id_within_range = df_assetid_byinv.loc[i].loc[prev_date:d]
        inv_uni = asset_id_within_range.unique().tolist()
        return inv_uni

    for (inv_id, date) in idx_inv_universe.to_flat_index():
        inv_uni = calc_past_quarters(inv_id, date)
        df_inv_universe.loc[(inv_id, date), 'inv_universe'] = inv_uni

    df_inv_universe = df_inv_universe.assign(uni_size=lambda x: x['inv_universe'].apply(len))
    log_inv_universe(df_inv_universe)
    return df_inv_universe


def create_equal_allocation(df_inv_universe: pd.DataFrame, df_aum_binned: pd.DataFrame) -> pd.DataFrame:
    df_equal_alloc = pd.merge(
        left=df_inv_universe,
        right=df_aum_binned,
        how='inner',
        left_index=True,
        right_index=True)
    
    return (df_equal_alloc
            .assign(allocation=lambda x: x['aum'] / x['uni_size'])
            .explode('inv_universe')
            .rename(columns={'inv_universe':'asset_id'})
            .set_index('asset_id', append=True))


def create_total_allocation(df: pd.DataFrame) -> pd.DataFrame:
    return (df['allocation']
            .groupby('asset_id')
            .sum())


def create_instrument(df_inv_universe: pd.DataFrame, df_aum_binned: pd.DataFrame) -> pd.DataFrame:
    df_equal_allocation = create_equal_allocation(df_inv_universe, df_aum_binned)
    total_allocation = create_total_allocation(df_equal_allocation)
    df_instrument = df_equal_allocation.assign(
        iv_me=lambda x: total_allocation - x['allocation'],
        ln_iv_me=lambda x: np.log(x['iv_me'])).dropna()
    log_instrument(df_instrument)
    return df_instrument


def calc_holding_weights(df_instrument: pd.DataFrame, df_inside_binned: pd.DataFrame, df_holding_factor: pd.DataFrame) -> pd.DataFrame:
    df_merged = pd.merge(
        left=df_instrument,
        right=df_inside_binned,
        how='outer',
        left_index=True,
        right_index=True).combine_first(df_holding_factor)
    
    # df_merged = pd.merge(
    #     left=df_merged,
    #     right=df_asset,
    #     how='left',
    #     on=['date', 'asset_id', 'prc', 'shrout', 'b_mkt', 'b_smb', 'b_hml'])

    df_model = (df_merged
                .assign(
                    weight=lambda x: np.log(x['holding'] / x['aum']),
                    ln_weight=lambda x: np.log(x['weight']),
                    ln_out_weight=lambda x: np.log(x['out_weight']),
                    ln_rweight=lambda x: x['ln_weight'] - x['ln_out_weight'],
                    rweight=lambda x: np.exp(x['ln_rweight']),
                    cons=lambda x: x['ln_rweight'].groupby(['inv_id', 'date']).transform('mean')))
    
    log_holding_weights(df_model)
    return df_model

In [77]:
# Estimation

def momcond(params, exog):
    bound = 0.999
    exog = exog.T
    
    beta_ln_me = params[0]
    beta_characteristics = params[1:]
    
    ln_me = exog[0]
    characteristics = exog[1:-1]
    out_weight = exog[-1]
    
    ln_me_term = (bound - np.exp(-1 * beta_ln_me)) * ln_me
    characteristics_term = np.dot(beta_characteristics, characteristics)
    pred_weight = np.exp(-1 * (ln_me_term + characteristics_term))
    
    return pred_weight * out_weight


def estimate_model(df_model: pd.DataFrame):
    idx = df_model.index
    cols = ['beta_ln_me', 'beta_cons', 'beta_b_mkt', 'beta_b_smb', 'beta_b_hml']
    df_params = pd.DataFrame(
        index=idx, 
        columns=cols)

    for (inv_id, date) in idx.to_flat_index():
        data = df_model.loc[(inv_id, date)]
        exog = data[['ln_me', 'b_mkt', 'b_smb', 'b_hml', 'out_weight']]
        instrument = data[['iv_me', 'b_mkt', 'b_smb', 'b_hml', 'out_weight']]
        endog = np.ones(exog.shape[0])

        model = gmm.NonlinearIVGMM(endog=endog, exog=exog, instrument=instrument, func=momcond, k_moms=1)
        w0inv = np.dot(instrument.T, instrument) / len(endog)
        start_params = np.ones(exog.shape[1])
        result = model.fit(start_params=start_params, maxiter=1000, inv_weights=w0inv, optim_method='hac')
        print(result.summary(yname='Demand System for Asset Pricing', xname=cols))
        df_params.loc[(inv_id, date)] = result.params
        # TODO finish momcond and sending data
        # TODO fix param recording/printing
        # TODO graphs/tables
        
    return df_params

In [92]:
# Log

def log_import_s12(df_s12: pd.DataFrame):
    dateindex_s12 = df_s12['rdate']
    print('Imported s12')
    print('Number of holdings:  ', len(df_s12))
    print('Earliest date:  ', min(dateindex_s12))
    print('Latest date:  ', max(dateindex_s12))


def log_import_s12type5(df_s12type5: pd.DataFrame):
    dateindex_s12type5 = df_s12type5['fdate']
    print()
    print('Imported s12type5')
    print('Number of holdings:  ', len(df_s12type5))
    print('Earliest date:  ', min(dateindex_s12type5))
    print('Latest date:  ', max(dateindex_s12type5))


def log_import_s34(df_s34: pd.DataFrame):
    dateindex_s34 = df_s34['rdate']
    print()
    print('Imported s34')
    print('Number of holdings:  ', len(df_s34))
    print('Earliest date:  ', min(dateindex_s34))
    print('Latest date:  ', max(dateindex_s34))


def log_import_ffm(df_ffm: pd.DataFrame):
    dateindex_ffm = df_ffm['dateff']
    print()
    print('Imported ffm')
    print('Number of dates:  ', len(df_ffm))
    print('Earliest date:  ', min(dateindex_ffm))
    print('Latest date:  ', max(dateindex_ffm))


def log_import_beta(df_beta: pd.DataFrame):
    dateindex_ffm = df_beta['DATE']
    print()
    print('Imported ffm')
    print('Number of dates:  ', len(df_beta))
    print('Earliest date:  ', min(dateindex_ffm))
    print('Latest date:  ', max(dateindex_ffm))


def log_import_security(df_security: pd.DataFrame):
    dateindex_security = df_security['datadate']
    print()
    print('Imported security')
    print('Number of holdings:  ', len(df_security))
    print('Earliest date:  ', min(dateindex_security))
    print('Latest date:  ', max(dateindex_security))


def log_import_fundamental(df_fundamental: pd.DataFrame):
    dateindex_fundamental = df_fundamental['datadate']
    print()
    print('Imported fundamental')
    print('Number of holdings:  ', len(df_fundamental))
    print('Earliest date:  ', min(dateindex_fundamental))
    print('Latest date:  ', max(dateindex_fundamental))


def log_clean_s12(df_s12_clean: pd.DataFrame):
    dateindex_s12 = df_s12_clean.index.get_level_values('date')
    print('Cleaned s12')
    print('Number of holdings:  ', len(df_s12_clean))
    print('Earliest date:  ', min(dateindex_s12))
    print('Latest date:  ', max(dateindex_s12))


def log_clean_s12type5(df_s12type5_clean: pd.DataFrame):
    dateindex_s12type5 = df_s12type5_clean.index.get_level_values('date')
    print()
    print('Cleaned s12type5')
    print('Number of firm/dates:  ', len(df_s12type5_clean))
    print('Earliest date:  ', min(dateindex_s12type5))
    print('Latest date:  ', max(dateindex_s12type5))


def log_clean_s34(df_s34_clean: pd.DataFrame):
    dateindex_s34 = df_s34_clean.index.get_level_values('date')
    print()
    print('Cleaned s34')
    print('Number of holdings:  ', len(df_s34_clean))
    print('Earliest date:  ', min(dateindex_s34))
    print('Latest date:  ', max(dateindex_s34))


def log_clean_ffm(df_ffm_clean: pd.DataFrame):
    dateindex_ffm = df_ffm_clean.index.get_level_values('date')
    print()
    print('Cleaned ffm')
    print('Number of dates:  ', len(df_ffm_clean))
    print('Earliest date:  ', min(dateindex_ffm))
    print('Latest date:  ', max(dateindex_ffm))


def log_clean_beta(df_beta_clean: pd.DataFrame):
    dateindex_beta = df_beta_clean.index.get_level_values('date')
    print()
    print('Cleaned beta')
    print('Number of dates:  ', len(df_beta_clean))
    print('Earliest date:  ', min(dateindex_beta))
    print('Latest date:  ', max(dateindex_beta))


def log_clean_security(df_security_clean: pd.DataFrame):
    dateindex_security = df_security_clean.index.get_level_values('date')
    print()
    print('Cleaned security')
    print('Number of asset/dates:  ', len(df_security_clean))
    print('Earliest date:  ', min(dateindex_security))
    print('Latest date:  ', max(dateindex_security))


def log_clean_fundamental(df_fundamental_clean: pd.DataFrame):
    dateindex_fundamental = df_fundamental_clean.index.get_level_values('date')
    print()
    print('Cleaned fundamental')
    print('Number of asset/dates:  ', len(df_fundamental_clean))
    print('Earliest date:  ', min(dateindex_fundamental))
    print('Latest date:  ', max(dateindex_fundamental))


def log_holding_merge(df_merged: pd.DataFrame):
    print('Merged s12 and s34')
    print('Number of holdings:  ', len(df_merged))
    print()


def log_asset_merge(df_merged: pd.DataFrame):
    print('Merged assets and factors')
    print('Number of assets/dates:  ', len(df_merged))
    print()


def log_holding_factor_merge(df_merged: pd.DataFrame):
    print('Merged holdings and factors')
    print('Number of assets/dates:  ', len(df_merged))
    print()


def log_household_sector(df_household: pd.DataFrame):
    print('Created household sector')
    print('Number of holdings:  ', len(df_household))


def log_outside_asset(df_holding: pd.DataFrame):
    print('Created outside asset')
    print('Number of holdings:  ', len(df_holding))


def log_inv_aum(df_inv_aum: pd.DataFrame):
    print('Calculated investor AUM')
    print(df_inv_aum.describe())


def log_bins(df_inside_binned: pd.DataFrame, df_aum_binned: pd.DataFrame):
    print('Binned investors')
    print('Number of investors:  ', len(df_inside_binned.index.get_level_values(0).unique()))
    print('Number of investors:  ', len(df_aum_binned.index.get_level_values(0).unique()))


def log_inv_universe(df_inv_uni: pd.DataFrame):
    print('Created investment universe')
    print(df_inv_uni.describe())


def log_instrument(df_instrument: pd.DataFrame):
    print('Created market equity instrument')
    print(df_instrument.describe())
    print()


def log_holding_weights(df_model: pd.DataFrame):
    print('Calculated holding weights')
    print(df_model.describe())


def log_inv_groups(df_group: pd.DataFrame):
    print('Partitioned investors')
    print('Number of groups:  ', len(df_group.index.get_level_values(0)))


def log_results(result):
    print(result.summary(
        yname='log portfolio weight',
        xname=['log market equity', 'market return', 'high minus low', 'small minus big']))
    print()

In [7]:
path = 'data/'
output = 'output/'
start_date = pd.Period('2012-01', freq='M')
end_date = pd.Period('2017-12', freq='M')


print('\n---------------Starting Imports---------------------------\n')
dfs = load_wrds(path, start_date, end_date)

print('\n---------------Starting Cleaning---------------------------\n')
df_s12_clean, df_s12type5_clean, df_s34_clean, df_ffm_clean, df_beta_clean, df_security_clean, df_fundamental_clean = clean_imports(
    *dfs,
    start_date,
    end_date
)

df_s12_clean.to_csv(os.path.join(output, 'df_s12_clean.csv'))
df_s12type5_clean.to_csv(os.path.join(output, 'df_s12type5_clean.csv'))
df_s34_clean.to_csv(os.path.join(output, 'df_s34_clean.csv'))
df_ffm_clean.to_csv(os.path.join(output, 'df_ffm_clean.csv'))
df_beta_clean.to_csv(os.path.join(output, 'df_beta_clean.csv'))
df_security_clean.to_csv(os.path.join(output, 'df_security_clean.csv'))
df_fundamental_clean.to_csv(os.path.join(output, 'df_fundamental_clean.csv'))

print('\n---------------Merging Assets/Factors---------------------------\n')
df_asset = merge_assets_factors(df_security_clean, df_beta_clean)
df_asset.to_csv(os.path.join(output, 'df_asset.csv'))

print('\n---------------Merging s12/s34 Holdings---------------------------\n')
df_merged = match_fund_manager(df_s12_clean, df_s34_clean, df_s12type5_clean)
df_merged.to_csv(os.path.join(output, 'df_merged.csv'))


---------------Starting Imports---------------------------

Imported s12
Number of holdings:   30987
Earliest date:   2014-06-30
Latest date:   2017-12-31

Imported s12type5
Number of holdings:   572518
Earliest date:   12/31/1994
Latest date:   9/30/2022

Imported s34
Number of holdings:   4841
Earliest date:   2012-03-31
Latest date:   2017-12-31

Imported ffm
Number of dates:   72
Earliest date:   2012-01-31
Latest date:   2017-12-29

Imported ffm
Number of dates:   432458
Earliest date:   2012-01-31
Latest date:   2017-12-29

Imported security
Number of holdings:   402452
Earliest date:   2012-01-31
Latest date:   2017-12-31

Imported fundamental
Number of holdings:   130615
Earliest date:   2012-06-30
Latest date:   2018-05-31

---------------Starting Cleaning---------------------------
Cleaned s12
Number of holdings:   27327
Earliest date:   2014-06
Latest date:   2017-12

Cleaned s12type5
Number of firm/dates:   49372
Earliest date:   2012-03
Latest date:   2017-12

Cleaned s34

In [40]:
print('\n---------------Merging Holdings/Factors---------------------------\n')
df_holding_factor = merge_holding_factor(df_merged, df_asset)
df_holding_factor.to_csv(os.path.join(output, 'df_holding_factor.csv'))

print('\n---------------Partitioning Outside Asset---------------------------\n')
df_inside, df_outside = create_outside_asset(df_holding_factor)
df_inside.to_csv(os.path.join(output, 'df_inside.csv'))
df_outside.to_csv(os.path.join(output, 'df_outside.csv'))

print('\n---------------Creating Household Sector---------------------------\n')
df_household = create_household_sector(df_inside)
df_household.to_csv(os.path.join(output, 'df_household.csv'))


---------------Merging Holdings/Factors---------------------------
Merged holdings and factors
Number of assets/dates:   4809


---------------Partitioning Outside Asset---------------------------

Created outside asset
Number of holdings:   52

---------------Creating Household Sector---------------------------

Created household sector
Number of holdings:   2500


In [230]:
print('\n---------------Calculating Investor AUM---------------------------\n')
df_inv_aum = calc_inv_aum(df_inside, df_outside)
df_inv_aum.to_csv(os.path.join(output, 'df_inv_aum.csv'))

print('\n---------------Pooling Investors By Type/Size---------------------------\n')
df_inside_binned, df_aum_binned = bin_concentrated_inv(df_inside, df_inv_aum)
df_inside_binned.to_csv(os.path.join(output, 'df_inside_binned.csv'))
df_aum_binned.to_csv(os.path.join(output, 'df_aum_binned.csv'))

print('\n---------------Tracking Investment Universe---------------------------\n')
df_inv_universe = calc_inv_universe(df_inside_binned)
df_inv_universe.to_csv(os.path.join(output, 'df_inv_universe.csv'))

print('\n---------------Calculating Instrument---------------------------\n')
df_instrument = create_instrument(df_inv_universe, df_aum_binned)
df_instrument.to_csv(os.path.join(output, 'df_instrument.csv'))


---------------Calculating Investor AUM---------------------------

Calculated investor AUM
             in_aum   n_holding   typecode       out_aum           aum
count  5.300000e+01   49.000000  49.000000  5.300000e+01  5.300000e+01
mean   5.978051e+08   58.734694   4.204082  2.687144e+08  8.665195e+08
std    6.988388e+08   40.906792   0.407206  2.948406e+08  9.017440e+08
min    0.000000e+00    1.000000   4.000000  0.000000e+00  1.230994e+07
25%    4.529729e+07   38.000000   4.000000  4.904507e+07  9.782133e+07
50%    6.837648e+07   55.000000   4.000000  6.282391e+07  1.280629e+08
75%    1.185759e+09   75.000000   4.000000  4.815766e+08  1.662714e+09
max    1.882495e+09  159.000000   5.000000  1.363582e+09  2.329958e+09

---------------Pooling Investors By Type/Size---------------------------

Binned investors
Number of investors:   7
Number of investors:   7

---------------Tracking Investment Universe---------------------------

Created investment universe
         uni_size
count  

C:\Users\wyatt\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [248]:
print('\n---------------Calculating Holding Weights---------------------------\n')
df_model = calc_holding_weights(df_instrument, df_inside_binned, df_holding_factor)
df_model.to_csv(os.path.join(output, 'df_model.csv'))

print('\n---------------Estimating Demand System---------------------------\n')
df_results = estimate_model(df_model)
df_results.to_csv(os.path.join(output, 'df_results.csv'))

print('\n---------------Finished---------------------------\n')


---------------Calculating Holding Weights---------------------------

Calculated holding weights
         allocation           aum        b_hml        b_mkt        b_smb  \
count  4.085000e+03  4.085000e+03  3246.000000  3246.000000  3246.000000   
mean   1.060272e+07  1.487445e+09    -0.004963     1.026580     0.033087   
std    6.326001e+06  8.677811e+08     0.643913     0.396848     0.599474   
min    1.460038e+06  4.057274e+07    -3.231900    -0.288800    -3.833400   
25%    2.574245e+06  1.289215e+08    -0.351300     0.757850    -0.385150   
50%    1.027838e+07  2.044520e+09    -0.061350     1.000500    -0.012800   
75%    1.679680e+07  2.143966e+09     0.286775     1.255175     0.361850   
max    4.909220e+07  2.329958e+09     7.200200     3.101200     2.568100   

            holding        in_aum    in_weight         iv_me     ln_iv_me  \
count  5.168000e+03  4.085000e+03  4085.000000  4.085000e+03  4085.000000   
mean   8.987703e+06  1.124058e+09     0.701074  2.053710e+08  

C:\Users\wyatt\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\wyatt\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\wyatt\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\wyatt\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


ValueError: too many values to unpack (expected 2)

In [249]:
df_asset

prc       shrout   b_mkt   b_smb   b_hml
date    asset_id                                             
2012-07 00095710  18.600   54201000.0  0.8011  1.0778  0.5065
        00103110  46.980    5524000.0  1.0750 -0.4055  1.0752
        00160010   6.720    3808000.0  0.8258  0.5093  0.5514
        00215Q10  19.720   81267000.0  0.8311  2.1696 -0.3116
        00289620  33.800   82567000.0  1.7054 -0.6199 -0.0366
...                  ...          ...     ...     ...     ...
2017-12 Y8564M10  20.150   79627000.0  1.1677  0.2662  1.2556
        Y8564W10   9.320   89127000.0  1.5039  1.3498  1.8666
        Y8565J10   2.360  410045000.0  2.6957  0.0852  1.3155
        Y8565N30   1.400  268202000.0  2.1212  0.7123 -0.5588
        Y8897Y23   0.251   89236000.0  1.2306  1.5504  2.2723

[98923 rows x 5 columns]